<a href="https://colab.research.google.com/github/evbevz/epoxy-supervisor/blob/main/Epoxy_Supervisor_Train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

In [1]:
%pip install ultralytics
%pip install -U albumentations
import ultralytics
ultralytics.checks()

Ultralytics 8.3.65 🚀 Python-3.11.11 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 31.1/112.6 GB disk)


# Git Clone?

In [2]:
# Задел на будущее, чтобы потом туда выкладывать обученные модели (веса)
!git clone https://github.com/evbevz/epoxy-supervisor.git

Cloning into 'epoxy-supervisor'...
remote: Enumerating objects: 479, done.
remote: Counting objects: 100% (246/246), done.
remote: Compressing objects: 100% (168/168), done.
remote: Total 479 (delta 168), reused 111 (delta 78), pack-reused 233 (from 2)
Receiving objects: 100% (479/479), 232.63 MiB | 23.31 MiB/s, done.
Resolving deltas: 100% (230/230), done.


# Load Dataset

In [3]:
# Download Epoxy Level Dataset
import torch
torch.hub.download_url_to_file('http://robot.bevz.space/dataset/epoxy-level-1-1774-new.zip', 'tmp.zip')
!unzip -q tmp.zip -d datasets && rm tmp.zip  # unzip

100%|██████████| 3.76G/3.76G [06:37<00:00, 10.2MB/s]


In [4]:
torch.hub.download_url_to_file('http://robot.bevz.space/dataset/epoxy-level-2-1140-new.zip', 'tmp.zip')
!unzip -q tmp.zip -d datasets && rm tmp.zip  # unzip

100%|██████████| 1.67G/1.67G [02:54<00:00, 10.2MB/s]


In [5]:
torch.hub.download_url_to_file('http://robot.bevz.space/dataset/epoxy-level-3-2640-new.zip', 'tmp.zip')
!unzip -q tmp.zip -d datasets && rm tmp.zip  # unzip
!touch /content/datasets/.gitignore

100%|██████████| 5.07G/5.07G [09:03<00:00, 10.0MB/s]


In [7]:
!unzip -q /content/epoxy-supervisor/EpoxyLevelDatasetsAnnotations.zip -d datasets

In [10]:
!mkdir -p runs/pose/train
!unzip -q /content/epoxy-supervisor/2025-01-21_1.train.zip -d runs/pose/train

# Train

In [ ]:
#@title Prepare logger
# Чтобы можно было наблюдать за ходом обучения в режиме реального времени, нужно загрузить TensorBoard до начала обучения модели с помощью команд:
# Подготовить TensorBoard к работе
%load_ext tensorboard
# Запустить TensorBoard
%tensorboard --logdir runs

In [ ]:
#@title Train YOLO11n-pose
#!yolo train model=yolo11n-pose.pt data=datasets/epoxy-level-1774/data.yaml epochs=200 imgsz=640 patience=100 save_period=5 cache=ram single_cls=True augment=True
#!yolo train model=yolo11n-pose.pt data=datasets/epoxy-level-1774/data.yaml epochs=300 imgsz=640 patience=100 save_period=5 cache=ram single_cls=True augment=True hsv_h=0.7 hsv_s=0.7 hsv_v=0.7 degrees=10 shear=10
#!yolo train model=yolo11n-pose.pt data=datasets/epoxy-level-1774/data.yaml epochs=300 imgsz=640 patience=100 save_period=5 cache=ram single_cls=True augment=True hsv_h=0.7 hsv_s=0.7 hsv_v=0.7 degrees=10 shear=10
#!yolo train model=/content/epoxy-supervisor/weights/epoxy-supervisor.20241221.pt data=datasets/epoxy-level-1140/data.yaml epochs=8000 imgsz=640 patience=100 save_period=100 cache=ram single_cls=True hsv_h=0.7 hsv_s=0.7 hsv_v=0.7 degrees=10 shear=10
#!yolo train model=/content/epoxy-supervisor/weights/epoxy-supervisor.20241225.pt data=datasets/epoxy-level-2640/data.yaml epochs=1100 imgsz=640 patience=100 save_period=100 cache=ram single_cls=True batch=32
#!yolo train model=/content/runs/pose/train/weights/last.pt data=/content/datasets/data.all.yaml epochs=1100 imgsz=640 patience=100 save_period=100 cache=ram single_cls=True batch=48 optimizer=AdamW
!yolo pose train model=yolo11s-pose.pt data=/content/datasets/el.all.data.yaml epochs=500 imgsz=640 patience=100 save_period=100 cache=ram batch=48 hsv_h=0.7 hsv_s=0.7 hsv_v=0.7 degrees=10 shear=10

In [ ]:
#@title Continue Train Model
# Планировал для того, чтобы продолжать обучение модели с последнего шага, но пока не до конца разобрался с параметрами.
# Почему то выдает ошибку.
#!yolo train resume model=runs/pose/train/weights/best.pt data=datasets/epoxy-level-1774/data.yaml epochs=10 imgsz=640
#!yolo train model=runs/pose/train/weights/last.pt data=datasets/epoxy-level-1774/data.yaml epochs=100 imgsz=640 patience=30 save_period=5 cache=ram single_cls=True augment=True
#!yolo train model=runs/pose/train/weights/best.pt data=datasets/epoxy-level-1774/data.yaml epochs=100 imgsz=640 patience=30 save_period=5 cache=ram single_cls=True augment=True hsv_h=0.7 hsv_s=0.7 hsv_v=0.7 degrees=10 shear=10
#!yolo train model=runs/pose/train2/weights/last.pt data=datasets/epoxy-level-1774/data.yaml epochs=100 imgsz=640 patience=50 save_period=5 cache=ram single_cls=True augment=True hsv_h=0.7 hsv_s=0.7 hsv_v=0.7 degrees=10 shear=10
#!yolo train model=runs/pose/train/weights/last.pt data=datasets/epoxy-level-1140/data.yaml epochs=400 imgsz=640 patience=100 save_period=10 cache=ram single_cls=True batch=0.7
#!yolo resume train model=runs/pose/train/weights/last.pt data=/content/datasets/el.all.data.yaml epochs=500 imgsz=640 patience=100 save_period=100 cache=ram single_cls=True batch=16
!yolo resume pose train model=runs/pose/train/weights/last.pt data=/content/datasets/el.all.data.yaml epochs=700 imgsz=640 patience=100 save_period=100 cache=ram batch=48

# Export

In [ ]:
import shutil
# Create a ZipFile
#!rm -rf /content/runs/pose/predict*
shutil.make_archive('/content/epoxy-supervisor/2025-01-21_1.train', 'zip', 'runs/pose/train')


In [ ]:
# Нужно будет добавить токен для авторизации, иначе не дает обратно заливать
%cd epoxy-supervisor
!git commit -m "Upload Train Data"
!git push

# Validate Epoxy Level Model

In [ ]:
# Validate fine-tune model
!yolo val model=/content/epoxy-supervisor/weights/epoxy-supervisor.20241224_2.pt data=datasets/epoxy-level-1774/data.yaml

In [ ]:
!yolo val model=/content/epoxy-supervisor/weights/epoxy-supervisor.20241224_2.pt data=datasets/epoxy-level-1140/data.yaml

In [ ]:
!yolo val model=/content/epoxy-supervisor/weights/epoxy-supervisor.20241224_2.pt data=datasets/epoxy-level-2640/data.yaml

# Predict Epoxy Level Model

In [ ]:
# Run inference on an image with Epoxy Level Model
#!yolo predict model=/content/runs/pose/train/weights/best.pt source='datasets/epoxy-level-1774/images/test/000074.png'
!yolo predict model=/content/epoxy-supervisor/weights/epoxy-supervisor.20241224_2.pt source='/content/datasets/epoxy-level-1774/images/test'

In [ ]:
!yolo predict model=/content/epoxy-supervisor/weights/epoxy-supervisor.20241224_2.pt source='/content/datasets/epoxy-level-1140/images/test'

In [ ]:
!yolo predict model=/content/epoxy-supervisor/weights/epoxy-supervisor.20241224_2.pt source='/content/datasets/epoxy-level-2640/images/test'

# Google Colab Session Keep-Alive



As of June 2023, here's the working javascript code for preventing Google colab from disconnecting due to inactivity.


```
 function keepAliveProgrammatically() {
  document.querySelector('colab-connect-button').shadowRoot.querySelector("#connect").click();
}
```
The function above is responsible for clicking the compute resources button It is clicked after every 6 seconds. The code snippet below is responsible for running the function.
```
const keepAliveProgrammaticallyInterval = setInterval(() => {
    keepAliveProgrammatically();
}, 600000);
```
To stop the code, use the code snippet below.
```
clearInterval(keepAliveProgrammaticallyInterval);
```